In [68]:
import os
import json
import GEOparse
import pandas as pd
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv()

True

In [69]:
gse= GEOparse.get_GEO(geo="GSE12277", destdir="./data/GEO")

03-Mar-2025 16:54:59 DEBUG utils - Directory ./data/GEO already exists. Skipping.
03-Mar-2025 16:54:59 INFO GEOparse - File already exist: using local version.
03-Mar-2025 16:54:59 INFO GEOparse - Parsing ./data/GEO/GSE12277_family.soft.gz: 
03-Mar-2025 16:54:59 DEBUG GEOparse - DATABASE: GeoMiame
03-Mar-2025 16:54:59 DEBUG GEOparse - SERIES: GSE12277
03-Mar-2025 16:54:59 DEBUG GEOparse - PLATFORM: GPL570
/home/bek/Desktop/iCog/semantic parsing/Bio_data_semantic_parsing/venv/lib/python3.12/site-packages/GEOparse/GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
03-Mar-2025 16:55:08 DEBUG GEOparse - SAMPLE: GSM308237
03-Mar-2025 16:55:08 DEBUG GEOparse - SAMPLE: GSM308239
03-Mar-2025 16:55:09 DEBUG GEOparse - SAMPLE: GSM308241
03-Mar-2025 16:55:11 DEBUG GEOparse - SAMPLE: GSM308242
03-Mar-2025 16:55:11 DEBUG GEOparse - SAMPLE: GSM308244
03-Mar-2025 16:55:12 DEBU

### Predicate generation for table entries

In [70]:
gsm= gse.gsms["GSM308246"]
data= gsm.table
data.head()

,ID_REF,VALUE,ABS_CALL,DETECTION P-VALUE
0,AFFX-BioB-5_at,115.4570,P,0.000581
1,AFFX-BioB-M_at,119.7700,P,0.000044
2,AFFX-BioB-3_at,84.8163,P,0.000095
3,AFFX-BioC-5_at,296.3510,P,0.000052
4,AFFX-BioC-3_at,470.6520,P,0.000044


In [71]:
sample_data=data.sample(100)
print(sample_data) 

            ID_REF       VALUE ABS_CALL  DETECTION P-VALUE
7439   207859_s_at    5.770820        A           0.303711
6136     206549_at    4.622190        A           0.904785
33374  233510_s_at   71.023100        P           0.000244
4863     205275_at    3.160390        A           0.805420
15361    215927_at    7.791850        A           0.665527
...            ...         ...      ...                ...
4641     205053_at  151.770000        P           0.000244
4374   204786_s_at   36.204000        P           0.000244
29017    229150_at    0.978748        A           0.725830
34759  234897_s_at    2.679120        A           0.725830
12482    213038_at   11.217800        A           0.601074

[100 rows x 4 columns]


In [72]:
columns = list(sample_data.columns)
print("Extracted Columns:", columns)


Extracted Columns: ['ID_REF', 'VALUE', 'ABS_CALL', 'DETECTION P-VALUE']


In [73]:
prompt = f"""
    You are a data standardization expert. Convert the following dataset column names into standardized predicate names:
    {columns}
    Return a JSON dictionary where keys are original column names and values are standardized predicate names.
    """

In [74]:
def openai_generate(messages: list, model: str = 'gpt-4', **generation_params):
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    completion = client.chat.completions.create(
        messages=messages,
        model=model,
        **generation_params
    )
    return completion

In [75]:
messages =[
    {'role':'system', 'content':"You are an expert in data standardization."},
    {'role':'user', 'content':prompt}
]

In [76]:
completion = openai_generate(messages, temperature=1.0)
predicate_mapping=completion.choices[0].message.content
print("Predicate Mapping:", predicate_mapping)

Predicate Mapping: {
    "ID_REF": "idReference",
    "VALUE": "value",
    "ABS_CALL": "absoluteCall",
    "DETECTION P-VALUE": "detectionPValue"
}


In [77]:
predicate_mapping = json.loads(predicate_mapping)

In [78]:
def generate_predicate_entry(row, predicate_mapping):
    predicate_values = []
    for original_col, predicate in predicate_mapping.items():
        value = row.get(original_col, "null")
        predicate_values.append(f'{predicate}({value if pd.notna(value) else "null"})')
    return f'GSM_Measurement(\n    {",\n    ".join(predicate_values)}\n)'


In [79]:
predicate_entries = sample_data.apply(lambda row: generate_predicate_entry(row, predicate_mapping), axis=1)

print("Example Predicate Output:")
print("\n".join(predicate_entries))



Example Predicate Output:
GSM_Measurement(
    idReference(207859_s_at),
    value(5.77082),
    absoluteCall(A),
    detectionPValue(0.303711)
)
GSM_Measurement(
    idReference(206549_at),
    value(4.62219),
    absoluteCall(A),
    detectionPValue(0.904785)
)
GSM_Measurement(
    idReference(233510_s_at),
    value(71.0231),
    absoluteCall(P),
    detectionPValue(0.000244141)
)
GSM_Measurement(
    idReference(205275_at),
    value(3.16039),
    absoluteCall(A),
    detectionPValue(0.80542)
)
GSM_Measurement(
    idReference(215927_at),
    value(7.79185),
    absoluteCall(A),
    detectionPValue(0.665527)
)
GSM_Measurement(
    idReference(1553432_s_at),
    value(3.81864),
    absoluteCall(A),
    detectionPValue(0.466064)
)
GSM_Measurement(
    idReference(207616_s_at),
    value(63.2889),
    absoluteCall(P),
    detectionPValue(0.000244141)
)
GSM_Measurement(
    idReference(1557217_a_at),
    value(0.0942226),
    absoluteCall(A),
    detectionPValue(0.884033)
)
GSM_Measure

In [80]:
with open("predicates.txt", "w") as file:
    file.write("\n".join(predicate_entries))